<h1>Pierwszy projekt ML</h1>

<h3>Problem regresji</h3>
W tym notatniku przejdziemy przez podstawowe kroki załadowania, obróbki i predykcji danych. 
Kolejność kroków wykonanych w tym notatniku :

1.) Analiza danych i działanie na obserwacjach
    - Import bibliotek i danych
    - Opis danych
    - Znalezienie korelacji między atrybutami
    - Pozbycie się odstających obserwacji
    - Wprowadzenie brakujących danych
    - Poprawienie atrybutów
    - Dodanie atrybutów
    
2.) Cechy statystyczne 
    - Skośność i kurtoza
    - Label Encoding
    - Transformacja i skalowanie danych
    - Wybór atrybutów
    - Principal Component Analysis
    
3.) Dobór modelu i jego ocena
    - Testowanie różnych modeli
    - Hiperparametryzacja
    - Łączenie modeli
    - Predykcja

<h1>1.) Analiza danych i działania na obserwacjach </h1>

<h1>Import bibliotek i danych</h1>

In [ ]:
import pandas as pd # processing danych
import numpy as np 
import matplotlib.pyplot as plt # wykresy
%matplotlib inline
import seaborn as sns # wizualizacja danych
color = sns.color_palette()
sns.set_style('darkgrid')
import scipy.stats as st # moduł statystyczny
pd.options.display.max_columns = None # show all columns
import missingno as msno # wizualizacje brakujących danych
import warnings # ignorowanie ostrzeżeń 
warnings.filterwarnings('ignore')

# Ładujemy dane testowe i treningowe do formatu DataFrame biblioteki Pandas

trainData = pd.read_csv("C:/Users/ksmoc/OneDrive/Workspace/PycharmProjects/ML_Projects/House_Pricing/data/train.csv")
testData = pd.read_csv("C:/Users/ksmoc/OneDrive/Workspace/PycharmProjects/ML_Projects/House_Pricing/data/test.csv")
trainData.drop(columns = 'Id', inplace =True)
y_train = trainData['SalePrice']

<h1>Wstępna charakterystyka danych</h1>
Spójrzmy na dane co mamy i jaki jest ich ogólny obraz

In [ ]:
# Kształt danych (wymiary tablic)
trainData.shape, testData.shape, y_train.shape

In [ ]:
# Pierwsze pięć kolumn zbioru
trainData.head()

In [ ]:
# Ostatnie pięć kolumn zbioru
trainData.tail()

In [ ]:
# Podstawowy opis zbioru danych
trainData.describe()

In [ ]:
# Atrybuty w zbiorze danych zawierające dane numeryczne 
numerical_features = trainData.select_dtypes(include=[np.number]) 
numerical_features.columns

# Atrybuty w zbiorze danych zawierające dane kategorialne
categorical_features = trainData.select_dtypes(include=[np.object])
categorical_features.columns

<h1>Znalezienie korelacji między atrybutami</h1>
Spojrzymy jak się rozkładają korelacje pomiędzy poszczególnymi atrybutami

In [ ]:
# Spróbujemy znaleźć atrybuty posiadające największą korelacje (czyli te od których najbardziej zależy) 
# szukana przez nas wartość 'SalePrice', posortowane od największej do najmniejszej.

correlation = numerical_features.corr()
print(correlation['SalePrice'].sort_values(ascending = False))

In [ ]:
# Mapa cieplna korelacji atrybutów numerycznych
f , ax = plt.subplots(figsize = (14,12))
plt.title('Korelacja atrybutów numerycznych',size=15)
sns.heatmap(correlation,square = True,  vmax=0.8)

- Widzimy dwa białe kwadraty (2,2 and 3,3) Które wskazują na wysoką korelację. Pierwsza grupa silnie skorelowanych atrybutów to 'TotalBsmtSF' i '1stFlrSF'. Druga grupa to 'GarageYrBlt', 'GarageCars' i 'GarageArea'. To oznacza wieloliniowość.
- Inne cztery białe kwadraty (1,1) wskazują na oczywistą korelację między 'GarageYrBlt' i 'YearBuilt' oraz między 'TotRmsAbvGrd' i 'GrLivArea'
- Ponadto z mapy cieplnej i poprzedniej oceny korelacji odczytujemy, że'GrLivArea', 'TotalBsmtSF', 'OverallQual', 'FullBath', 'TotRmsAbvGrd' oraz 'YearBuilt' są silnie skorelowane z 'SalePrice'

In [ ]:
# Zbliżenie mapy cieplnej najbardziej skorelowanych atrybutów
zoomedCorrelation = correlation.loc[['SalePrice','GrLivArea','TotalBsmtSF','OverallQual','FullBath','TotRmsAbvGrd','YearBuilt','1stFlrSF','GarageYrBlt','GarageCars','GarageArea'], ['SalePrice','GrLivArea','TotalBsmtSF','OverallQual','FullBath','TotRmsAbvGrd','YearBuilt','1stFlrSF','GarageYrBlt','GarageCars','GarageArea']]
f , ax = plt.subplots(figsize = (14,12))
plt.title('Korelacja atrybutów numerycznych',size=15)
sns.heatmap(zoomedCorrelation, square = True, linewidths=0.01, vmax=0.8, annot=True,cmap='viridis',
            linecolor="black", annot_kws = {'size':12})

Stwierdzamy, że:
- 'TotalBsmtSF' oraz '1stFlrSF' są silnie skorelowane
- 'TotRmsAbvGrd' oraz 'GrLivArea' są silnie skorelowane
- 'GarageCars' oraz 'GarageArea' są silnie skorelowane
- 'GarageYrBlt' oraz 'YearBuilt' są silnie skorelowane
- 'TotRmsAbvGrd' oraz 'GrLivArea' są silnie skorelowane
- 'OverallQual', 'GrLivArea' i 'TotRmsAbvGrd' są silnie skorelowane z 'SalePrice'

In [ ]:
# Wykresy dwóch zmiennych
sns.set()
cols = ['SalePrice','GrLivArea','TotalBsmtSF','OverallQual','FullBath','TotRmsAbvGrd','YearBuilt','1stFlrSF','GarageYrBlt','GarageCars','GarageArea']
sns.pairplot(trainData[cols],size = 2 ,kind ='scatter',diag_kind='kde')
plt.show()

- Widzimy, że 'SalePrice' wzrasta kwadratowo wraz z wzrostem wartości 'TotalBsmtSF', 'GrLivArea', '1stFlrSF'. Wnioskujemy z tego, że cena domu zwiększa się o kwadrat przyrostu powierzchni. Ponadto widzimy, że 'SalePrice' wzrasta wykładniczo wraz z 'OverallQual'.
- Ponadto z 'GrLivArea'-'1stFlSF' oraz '1stFlSF'-'TotalBsmSF' obserwujemy, że wszystkie punkty znajdują się powyżej lini funkcji tożsamościowej, co oznacza, że parter posiada większą powierzchnie niż którekolwiek z pięter, oraz, że pierwsze piętro jest większe niż piwnica.
- Podobne zjawisko zachodzi dla 'GarageYrBlt'-'YearBuilt' co ma sens ponieważ przeważnie najpierw budujemy dom a dopiero następnie garaż, jednakże zachodzą tutaj pewne wyjątki w naszym zbiorze danych.

<h1>Pozbycie się odstających obserwacji</h1>

Z poprzednich wykresów dwóch zmiennych widzimy kilka obserwacji odstających dla 'TotalBsmtSF', '1stFlrSF' oraz 'GrLivArea'. Skorzystamy z wykresu punktowego aby zobaczyć je bardziej dokładnie.

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(x = trainData.TotalBsmtSF,y = trainData.SalePrice)
plt.title('TotalBsmtSF', size = 15)
plt.figure(figsize=(7,5))
plt.scatter(x = trainData['1stFlrSF'],y = trainData.SalePrice)
plt.title('1stFlrSF', size = 15)
plt.figure(figsize=(7,5))
plt.scatter(x = trainData.GrLivArea,y = trainData.SalePrice)
plt.title('GrLivArea', size = 15)

In [ ]:
# Usuwanie obserwacji odstających
trainData.drop(trainData[trainData['TotalBsmtSF'] > 5000].index,inplace = True)
trainData.drop(trainData[trainData['1stFlrSF'] > 4000].index,inplace = True)
trainData.drop(trainData[(trainData['GrLivArea'] > 4000) & (trainData['SalePrice']<300000)].index,inplace = True)
trainData.shape

Ponieważ odrzuciliśmy tylko dwie obserwacje odstające oznacza to, że wszystkie trzy cechy dzieliły tą samą obserwację.

<h1>Wprowadzenie brakujących danych</h1>

Teraz przyjrzymy się brakujących danych w naszym zbiorze.
Będziemy korzystać z biblioteki msno (missingno). Msno zapewnia mały zestaw narzędzi do wizualizacji brakujących danych oraz funkcjonalności które pozwalają na szybkie wizualne podsumowanie kompletności danych albo jej braku w twoim zbiorze.

In [ ]:
# Wizualizowanie brakujących wartości dla atrybutów numerycznych z próby 200 obserwacji
msno.matrix(trainData.select_dtypes(include=[np.number]).sample(200))

In [ ]:
# Wizualizowanie procentu brakujących wartości spośród 10 najbardziej wybrakowanych atrybutów numerycznych
total = trainData.select_dtypes(include=[np.number]).isnull().sum().sort_values(ascending=False)
percent = (trainData.select_dtypes(include=[np.number]).isnull().sum()/trainData.select_dtypes(include=[np.number]).isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,join='outer', keys=['Missing Count', 'Missing Percentage'])
missing_data.index.name =' Numeric Feature'
missing_data.head(10)

We observe that 'LotFrontage', 'GarageYrBlt' and 'MasVnrArea' are the only one who have missing values

In [ ]:
# Wizualizacja brakujących danych z atrybutów o kategorialnych danych w próbie dwustu obserwacji.
msno.matrix(trainData.select_dtypes(include=[np.object]).sample(200))

In [ ]:
# Wizualizacja procentowego udziału brakujących obserwacji w top dziesięciu atrybutach zawierających brakujące dane kategorialne
total = trainData.select_dtypes(include=[np.object]).isnull().sum().sort_values(ascending=False)
percent = (trainData.select_dtypes(include=[np.object]).isnull().sum()/trainData.select_dtypes(include=[np.object]).isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1,join='outer', keys=['Missing Count', 'Missing Percentage'])
missing_data.index.name =' Numeric Feature'
missing_data.head(10)

Okazuje się, że 'PoolQC', 'MiscFeature', 'Alley', 'Fence' oraz 'FireplaceQu' posiadają znaczącą ilość brakujących danych (przynajmniej połowa obserwacji)

In [ ]:
# Wizualizacja wybrakowania według kolumny
msno.bar(trainData.sample(1000))

In [ ]:
# Mapa cieplna korelacji wybrakowania tz. jak bardzo obecność lub brak pewnej obserwacji wpływa na inną
msno.heatmap(trainData)

# -1 : jeżeli jedna obserwacja jest obecna drugiej na pewno nie ma
# 0 : obecność obserwacji lub jej brak nie ma wpływu na inną obserwację  
# 1 : jeżeli jedna obserwacja jest obecna druga na pewno też jest

In [ ]:
# Dendrogram kompletności obserwacji, pokazuje trendy korelacyjne między obserwacjami głębsze niż te wynikające z mapy cieplnej.
msno.dendrogram(trainData)

Liście klastra które są połączone ze sobą z zerową odległością, w pełni określają swoją wzajemną obecność: jedna obserwacja może być zawsze pusta jeżeli druga istnieje, lub obie mogą być zawsze obecne lub puste.

Rozpoczniemy od zastępowania brakujących danych w zbiorach testowych i treningowych. 
W tym celu najpierw połączymy je w jeden zbiór.

In [ ]:
# Powiążemy zbiory treningowy i testowy w jeden obiekt Dataframe
dataFull = pd.concat([trainData,testData],ignore_index=True)
dataFull.drop('Id',axis = 1,inplace = True)
dataFull.shape

In [ ]:
# Suma brakujących obserwacji w zależności od atrybutu
sumMissingValues = dataFull.isnull().sum()
sumMissingValues[sumMissingValues>0].sort_values(ascending = False)

In [ ]:
# Atrybuty numeryczne: zastępujemy zerem. Dlaczego akurat te?
for col in ['BsmtFullBath','BsmtHalfBath','BsmtUnfSF','TotalBsmtSF','GarageCars','BsmtFinSF2','BsmtFinSF1','GarageArea']:
    dataFull[col].fillna(0,inplace= True)

# Sprawdzamy czy udało nam się je zastąpić.
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Rozpoczniemy od atrybutów które posiadają mniej niż 5 brakujących obserwacji

In [ ]:
# Atrybuty kategorialne: zastępujemy modalną (najczęściej występującą wartością)
for col in ['MSZoning','Functional','Utilities','KitchenQual','SaleType','Exterior2nd','Exterior1st','Electrical']:
    dataFull[col].fillna(dataFull[col].mode()[0],inplace= True)

# Sprawdzamy czy udało nam się je zastąpić.
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

In [ ]:
# Przypisujemy atrybuty które posiadają więcej niż 5 brakujących obserwacji.

# Dane kategorialne: Zmieniamy wszystkie na "None"
for col in ['PoolQC','MiscFeature','Alley','Fence','FireplaceQu','GarageQual','GarageCond','GarageFinish','GarageType','BsmtExposure','BsmtCond','BsmtQual','BsmtFinType2','BsmtFinType1','MasVnrType']:
    dataFull[col].fillna('None',inplace = True)

# Sprawdzamy czy udało nam się je zastąpić.
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Ponieważ 'MasVnrArea' posiada tylko 23 brakujące obserwacje, możemy zastąpić je średnią dla kolumny.

In [ ]:
dataFull['MasVnrArea'].fillna(dataFull['MasVnrArea'].mean(), inplace=True)

# Sprawdzamy czy udało nam się je zastąpić.
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Bazując na mapie cieplnej korelacji wiemy, że 'GarageYrBlt' jest silnie skorelowane z 'YearBuilt'. Z tego powodu zastąpimy brakujące wartości medianami z 'YearBuilt'. 

Z tego względu iż atrybut 'YearBuilt' zawiera dane numeryczne musimy podzielić go na przedziały.

In [ ]:
# Dzielimy 'YearBuilt' na 10 przedziałów
dataFull['YearBuiltCut'] = pd.qcut(dataFull.YearBuilt,10)
# Zastąpienie brakujących obserwacji atrybutu 'GarageYrBlt' bazując na medianie atrybutu 'YearBuilt' 
dataFull['GarageYrBlt']= dataFull.groupby(['YearBuiltCut'])['GarageYrBlt'].transform(lambda x : x.fillna(x.median()))
# Rzutowanie typu na liczbowy (int)
dataFull['GarageYrBlt'] = dataFull['GarageYrBlt'].astype(int)
# Usuwamy kolumnę 'YearBuiltCut'
dataFull.drop('YearBuiltCut',axis=1,inplace=True)
# # Sprawdzamy czy udało nam się zastąpić brakujące obserwacje.
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Na podstawie mapy cieplnej korelacji wiemy że, 'LotFrontage' jest silnie skorelowane z 'LotArea' oraz 'Neighbourhood'. 
Dokonamy tego samego co w przypadku 'YearBuilt'

In [ ]:
# Dzielimy atrybut 'LotArea' na 10 przedziałów
dataFull['LotAreaCut'] = pd.qcut(dataFull.LotArea,10)

# Zastępujemy brakujące obserwacje atrybutu 'LotFrontage' opierając się na medianie atrybutów 'LotArea' oraz 'Neighbourhood'
dataFull['LotFrontage']= dataFull.groupby(['LotAreaCut','Neighborhood'])['LotFrontage'].transform(lambda x : x.fillna(x.median()))
dataFull['LotFrontage']= dataFull.groupby(['LotAreaCut'])['LotFrontage'].transform(lambda x : x.fillna(x.median()))

# Usuwamy kolumnę 'LotAreaCut'
dataFull.drop('LotAreaCut',axis=1,inplace=True)

# Sprawdzamy czy udało nam się zastąpić brakujące obserwacje.
dataFull.isnull().sum()[dataFull.isnull().sum()>0].sort_values(ascending = False)

Jedyne pozostałe brakujące obserwacje należą do atrybutu 'Sale price', który odzwierciedla liczbę obserwacji z testowego zbioru danych, które musimy przewidzieć.

<h1>Poprawianie atrybutów</h1>

Jeżeli spojrzymy na zmienne numeryczne zouważymy, że część z nich nie ma sensu aby była numeryczna, tak jak atrybuty związane z datami. Spójrzmy na nie w pliku opisującym dane i zobaczmy które powinnismy zmienic na kategorialne.

In [ ]:
dataFull.select_dtypes(include=[np.number]).columns

In [ ]:
# Konwersja zmiennych numerycznych na kategorialne
strCols = ['YrSold','YearRemodAdd','YearBuilt','MoSold','MSSubClass','GarageYrBlt']
for i in strCols:
    dataFull[i]=dataFull[i].astype(str)

<h1> Dodawanie atrybutów </h1>

Po pierwsze będziemy mapować zmienne kategorialne tak aby tworzyły swojego rodzaju ranking wyrażony w liczbach całkowitych.

In [ ]:
dataFull.select_dtypes(include=[np.object]).columns

In [ ]:
dataFull["oExterQual"] = dataFull.ExterQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
dataFull["oBsmtQual"] = dataFull.BsmtQual.map({'None':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
dataFull["oBsmtExposure"] = dataFull.BsmtExposure.map({'None':1, 'No':2, 'Av':3, 'Mn':3, 'Gd':4})
dataFull["oHeatingQC"] = dataFull.HeatingQC.map({'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})
dataFull["oKitchenQual"] = dataFull.KitchenQual.map({'Fa':1, 'TA':2, 'Gd':3, 'Ex':4})
dataFull["oFireplaceQu"] = dataFull.FireplaceQu.map({'None':1, 'Po':2, 'Fa':3, 'TA':4, 'Gd':5, 'Ex':6})
dataFull["oGarageFinish"] = dataFull.GarageFinish.map({'None':1, 'Unf':2, 'RFn':3, 'Fin':4})
dataFull["oPavedDrive"] = dataFull.PavedDrive.map({'N':1, 'P':2, 'Y':3})

Następnie dodamy kilka atrybutów numerycznych do siebie, tak aby stworzyć nowe atrybuty, które miały by sens.

In [ ]:
dataFull.select_dtypes(include=[np.number]).columns

In [ ]:
dataFull['HouseSF'] = dataFull['1stFlrSF'] + dataFull['2ndFlrSF'] + dataFull['TotalBsmtSF']
dataFull['PorchSF'] = dataFull['3SsnPorch'] + dataFull['EnclosedPorch'] + dataFull['OpenPorchSF'] + dataFull['ScreenPorch']
dataFull['TotalSF'] = dataFull['HouseSF'] + dataFull['PorchSF'] + dataFull['GarageArea']